# Clean

This notebook loads your assembled data and prepares it for modeling.

The final output is a `sup` that is fully ready for modeling and analysis.

Here's what happens in this notebook:
- We fill gaps in the data using reasonable assumptions
- We prepare the data for further analysis by marking clusters
- We process sales validity information
- We run our own sales scrutiny heuristic to make sure we only use trustworthy sales that reflect market value

These operations are necessary for modeling, but nevertheless inject a certain amount of subjectivity into the model, which is why we keep the results of the **Clean** notebook separate from those produced in the **Assemble** notebook.

In [10]:
# Change these as desired

# The slug of the locality you are currently working on
locality = "us-ky-louisville"

# Whether to print out a lot of stuff (can help with debugging) or stay mostly quiet
verbose = True

# Clear previous state for this notebook and start fresh
clear_checkpoints = True

# Set to true to have sales scrutiny drop sales rather than just flagging them
sales_scrutiny_drop_outliers = False    # Drop outlier sales in sales clusters
sales_scrutiny_drop_heuristics = True  # Drop sales that match suspicious metadata patterns

# 1. Basic setup

In [11]:
import init_notebooks
init_notebooks.setup_environment()
locality = init_notebooks.check_for_different_locality(locality)

Environment setup completed.


In [12]:
# import a bunch of stuff
from openavmkit.pipeline import (
    init_notebook,
    from_checkpoint,
    delete_checkpoints,
    write_checkpoint,
    read_pickle,
    load_settings,
    examine_sup,
    fill_unknown_values_sup,
    process_sales,
    mark_ss_ids_per_model_group_sup,
    mark_horizontal_equity_clusters_per_model_group_sup,
    run_sales_scrutiny,
    write_notebook_output_sup
)

In [13]:
init_notebook(locality)

locality = us-ky-louisville
base path = C:\Users\jacks\Documents\Non-Game Stuff\Programming\openavmkit\notebooks\pipeline
current path = C:\Users\jacks\Documents\Non-Game Stuff\Programming\openavmkit\notebooks\pipeline\data\us-ky-louisville


In [14]:
if clear_checkpoints:
    delete_checkpoints("2-clean")

# 2. Load data

In [15]:
settings = load_settings()

In [16]:
# load the data
sales_univ_pair = read_pickle("out/1-assemble-sup")

In [17]:
examine_sup(sales_univ_pair, settings)


EXAMINING UNIVERSE...

            FIELD                 TYPE     NON-ZERO    %    NON-NULL    %                    UNIQUE                 
============================== ========== ========== ===== ========== ===== ========================================
             LAND             
============================== ========== ========== ===== ========== ===== ========================================
------------------------------ ---------- ---------- ----- ---------- ----- ----------------------------------------
           NUMERIC            
------------------------------ ---------- ---------- ----- ---------- ----- ----------------------------------------
assr_land_value                                                              Float64      277,198   98%    282,400  100%                                         
depth_ft_1                                                                   float64      268,482   95%    282,400  100%                                         
depth

# 3. Fill unknowns

Modeling functions are unable to process null data, so you need to fill them in somehow.   
The goal is to **eliminate all gaps in your data,** at least for fields you intend to turn into modeling variables.

Consult the documentation for more details and best practices on filling unknown values.

In [18]:
# Fill holes in the data with sensible defaults
sales_univ_pair = fill_unknown_values_sup(sales_univ_pair, settings)

# 4. Clustering

We cluster all similar properties and give each cluster a unique ID.  
Later, we'll use these ID's whenever we want to run a horizontal equity study.


In [19]:
settings = load_settings()
sales_univ_pair = from_checkpoint("2-clean-00-horizontal-equity", mark_horizontal_equity_clusters_per_model_group_sup,
    {
        "sup": sales_univ_pair,
        "settings": settings,
        "verbose": verbose,
        "do_land_clusters": True,
        "do_impr_clusters": True
    }
)


Marking horizontal equity clusters...

Marking LAND horizontal equity clusters...

Marking IMPROVEMENT horizontal equity clusters...


## 5. Process sales

We process sales validity information to set all the right codes for later use.  
We calculate time trends for sales over time to generate time-adjusted sale prices.

In [20]:
sales_univ_pair = from_checkpoint("2-clean-01-process_sales", process_sales,
    {
        "sup": sales_univ_pair,
        "settings": load_settings(),
        "verbose": verbose
    }
)

Before univ merge len = 237834
After univ merge len = 237834
Using 78664 sales...
--> 4039 vacant sales
--> 74625 improved sales
--> 72269 valid for ratio study
--> 4039 valid for land ratio study
len before validate = 78664
Invalid sales validation filter disabled, skipping...
len after validate = 78664
len after hydrate = 78664
Applying time adjustment...


C:\Users\jacks\Documents\Non-Game Stuff\Programming\openavmkit\openavmkit\time_adjustment.py:431: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_impr["sale_price_per_impr_sqft"] = div_df_z_safe(
C:\Users\jacks\Documents\Non-Game Stuff\Programming\openavmkit\openavmkit\time_adjustment.py:434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_land["sale_price_per_land_sqft"] = div_df_z_safe(


--> Using period: Q
--> Crunching time adjustment...
--> Flattening time adjustment...


C:\Users\jacks\Documents\Non-Game Stuff\Programming\openavmkit\openavmkit\time_adjustment.py:230: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["sale_date"] = pd.to_datetime(df["sale_date"])


---->interpolating missing periods...
--> Time adjustment calculated for 2083 days.
len after enrich = 78664
len after clip = 78664


## 6. Scrutinize sales

We cluster all sales of similar properties in similar locations.  
We flag individual sales that are anomalously high or low for their local cluster.  
This helps us catch potentially invalid sales that slipped by the assessor's notice.  

In [21]:
sales_univ_pair = from_checkpoint("2-clean-02-sales-scrutiny", run_sales_scrutiny,
    {
        "sup": sales_univ_pair, 
        "settings": load_settings(), 
        "drop_cluster_outliers": sales_scrutiny_drop_outliers, 
        "drop_heuristic_outliers": sales_scrutiny_drop_heuristics, 
        "verbose": verbose
    }
)

Validating sales by heuristic, 78664 total sales
--> 22132 bad keys for heuristic: flag_dupe_date_price
--> 36 bad keys for heuristic: flag_false_vacant
Dropped 22162 invalid sales keys identified by heuristic

No manual exclusions file specified in settings, skipping manual exclusions
Processing model group: single_family
--> Flagged 7 vacant sales
--> Flagged 4205 improved sales
--> Unmarked sales before: 50877
--> Unmarked sales after: 46665
--> Marked 4212 new potentially invalid sales


# 7. Write out results

In [22]:
write_notebook_output_sup(sales_univ_pair, "2-clean")

...out/2-clean-sup.pickle
...out/look/2-clean-universe.parquet
...out/look/2-clean-sales.parquet
...out/look/2-clean-sales-hydrated.parquet


# 8. Look at it on a map!
- Take the files output in the previous step and put them in a map viewer like QGIS, ArcGIS, or Felt
- Look at them with your eyeballs
- Make sure the data looks correct
- If not, go back and fix it!
- Don't proceed to the next step until everything looks right